## DataFrames (dask)

In the previous notebook we manipulated and queried a large parquet file by splitting it up into many Pandas dataframes with the concurrent.futures API.  This was straightforward, but required us to be clever to implement various parallel algorithms.  Fortunately, for certain classes of algorithms various big data projects have already done this work for us.  Dealing with tabular data like the data in our NYC-Taxi dataset is well covered by many systems like SQL databases, Spark, and Dask.dataframes.

At the end of the last notebook we saw that we could use Pandas-like syntax to accomplish the same result with less programming.

In [ ]:
from gcsfs import GCSFileSystem
gcs = GCSFileSystem(token='cloud')

from dask.distributed import Client, progress
client = Client('schedulers:9000')
client.restart()

In [ ]:
import dask.dataframe as dd
df = dd.read_parquet('gcs://anaconda-public-data/nyc-taxi/2015.parquet').persist()


In [ ]:
df.passenger_count.sum().compute()

In [ ]:
df.passenger_count.mean().compute()

In [ ]:
df.passenger_count.value_counts().compute()

## Going further

In this notebook we will explore this API and this dataset further.  We will determine how well people tip based on the number of passengers in a cab.  To do this we will accomplish the following:

1.  Remove rides with zero fare
2.  Add a new column `tip_fraction` that is equal to the ratio of the tip to the fare
3.  Group by the `passenger_count` column and take the mean of the `tip_fraction` column.

### How to remove rows

In Pandas you can filter rows by a boolean expression like the following:

```python
df = df[df.name == 'Alice']
```

### How to make new columns

In Pandas you can create a new column using Python's setitem syntax like the following:

```python
df['z'] = df.x + df.y
```

### How to do groupby-aggregations

In Pandas you can do a groupby-aggregation by using the `groupby` method, followed by a column name an aggregation method like the following:

```python
df.groupby(df.name).value.mean()
```

### Dask.dataframes

In dask.dataframes all of these same commands still work.  The only difference is that at the end of your final line you will need to call `.compute()`

## Exercise

Use the operations above to find out how well New Yorkers tip based on the number of passengers in the cab.  Be sure to filter out rides with zero fare first to get a good result.

### Additional exercises

If you're already experienced with Pandas then we recommend also looking at the following questions:

1. How well do New Yorkers tip as a function of the hour of day and the day of the week?
2.  Investiate the `payment_type` column.  See how well each of the payment types correlate with the `tip_fraction`.  Did you find anything interesting?  Any guesses on what the different payment types might be?  If you're interested you may be able to find more information on the [NYC TLC's website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)
3.  How quickly can you get the data for a particular day of the year?  How about for a particular hour of that day?
